In [1]:
# Install required packages for Kaggle environment
!pip install -q packaging transformers datasets evaluate accelerate bitsandbytes rouge_score sacrebleu
!pip install -U peft

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 9.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1


In [2]:
# Install Unsloth-compatible TRL version
!pip install -q "trl<0.12.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 6.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 15.1 MB/s eta 0:00:00


In [3]:
# Install Unsloth
!pip install unsloth -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 10.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 15.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.7 MB/s eta 0:00:000:

In [4]:
import os
import torch
import pandas as pd
from datetime import datetime
from abc import ABC, abstractmethod
from datasets import load_dataset, Dataset
from unsloth import FastLanguageModel  # MUST import Unsloth FIRST!
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
import evaluate
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-01-06 12:31:49.410659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767702709.580828      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767702709.630637      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767702710.030948      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767702710.030984      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767702710.030987      55 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
# 1. STRATEGY INTERFACE
class FineTuningStrategy(ABC):
    @abstractmethod
    def setup_model(self, model_id, max_seq_length):
        """Initializes model and tokenizer based on the chosen strategy."""
        pass

# 2.1. CONCRETE LORA STRATEGY (Standard PEFT)
class LoRAStrategy(FineTuningStrategy):
    def setup_model(self, model_id, max_seq_length):
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )
        
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        # Standard fix for padding
        tokenizer.pad_token = tokenizer.eos_token
        
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto",
            dtype=torch.float16, 
            trust_remote_code=True
        )
        
        model = prepare_model_for_kbit_training(model)
        
        # Optimized for Llama architectures by including MLP layers
        lora_config = LoraConfig(
            r=16, 
            lora_alpha=32,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", 
                            "gate_proj", "up_proj", "down_proj"],
            lora_dropout=0.05, 
            bias="none", 
            task_type="CAUSAL_LM"
        )
        model = get_peft_model(model, lora_config)
        return model, tokenizer, lora_config

# 2.2. CONCRETE Unsloth STRATEGY (Highly Optimized)
class UnslothStrategy(FineTuningStrategy):
    def setup_model(self, model_id, max_seq_length):
        # 1. Load Model and Tokenizer via Unsloth
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = model_id,
            max_seq_length = max_seq_length,
            dtype = None, # Auto-detects (Bfloat16 for Ampere+, Float16 for T4)
            load_in_4bit = True,
        )

        # 2. Apply Unsloth-optimized LoRA
        model = FastLanguageModel.get_peft_model(
            model,
            r = 16, 
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", 
                              "gate_proj", "up_proj", "down_proj"],
            lora_alpha = 32,
            lora_dropout = 0, # Unsloth optimization: 0 is faster
            bias = "none",    
            use_gradient_checkpointing = "unsloth", 
            random_state = 3407,
        )
        
        # 3. FIX TOKENIZER (CRITICAL for Llama 3.1)
        # Llama 3.1 uses <|end_of_text|> but often needs <|eot_id|> for Instruct.
        # We ensure the tokenizer recognizes the end of the string to prevent loops.
        EOS_TOKEN = tokenizer.eos_token 
        tokenizer.pad_token = tokenizer.eos_token
        
        # Force model config to match tokenizer for SFTTrainer compatibility
        if hasattr(model, 'config'):
            model.config.pad_token_id = tokenizer.pad_token_id
            model.config.eos_token_id = tokenizer.eos_token_id
            
        return model, tokenizer, None

In [6]:
import pandas as pd
from datasets import Dataset

# 3. DATASET PROCESSOR (Final Verified Version)
class DatasetProcessor:
    def __init__(self, file_path, tokenizer):
        self.file_path = file_path
        self.tokenizer = tokenizer
        self.max_seq_limit = 2048

    def format_prompt(self, row):
        # We ensure the Llama 3 special tokens are used correctly
        # The prompt ends with the answer for SFT training
        text = (
            "<|begin_of_text|>"
            "<|start_header_id|>system<|end_header_id|>\n\n"
            "আপনি একজন সহানুভূতিশীল বাঙালি মানসিক স্বাস্থ্য পরামর্শদাতা।<|eot_id|>"
            "<|start_header_id|>user<|end_header_id|>\n\n"
            f"বিষয়: {row['Topics']} | {row['Question-Title']}\n"
            f"{row['Questions']}<|eot_id|>"
            "<|start_header_id|>assistant<|end_header_id|>\n\n"
            f"{row['Answers']}<|eot_id|>"
        )
        return {"text": text}

    def process_and_get_max_len(self):
        df = pd.read_csv(self.file_path)
        # Ensure we only process if columns exist
        required_cols = ['Topics', 'Question-Title', 'Questions', 'Answers']
        if not all(col in df.columns for col in required_cols):
            raise ValueError(f"CSV must contain: {required_cols}")

        sample_size = min(20000, len(df)) # Keep at 200 for your current test
        df_sampled = df.sample(n=sample_size, random_state=42).reset_index(drop=True)
        raw_ds = Dataset.from_pandas(df_sampled)

        def filter_and_calculate(example):
            formatted = self.format_prompt(example)
            tokens = self.tokenizer.encode(formatted["text"], add_special_tokens=False)
            ans_tokens = self.tokenizer.encode(str(example['Answers']), add_special_tokens=False)
            return {
                "text": formatted["text"],
                "total_len": len(tokens),
                "answer_len": len(ans_tokens)
            }

        processed_ds = raw_ds.map(filter_and_calculate)
        valid_ds = processed_ds.filter(lambda x: x["total_len"] <= self.max_seq_limit)
        
        max_answer_len = max(valid_ds["answer_len"]) if len(valid_ds) > 0 else 128
        
        # Split Logic (90/9/1 for small sample)
        train_test = valid_ds.train_test_split(test_size=0.10, seed=42)
        test_val = train_test['test'].train_test_split(test_size=0.1, seed=42)

        return train_test['train'], test_val['train'], test_val['test'], self.max_seq_limit, max_answer_len

In [7]:

# 4. EXPERIMENT LOGGER (As per Requirements 2.1)
class ExperimentLogger:
    """
    Logs experiment configurations, training metrics, and evaluation results.
    Ensures that results like Bleu, RougeL, and Perplexity are stored for analysis.
    """
    def log_to_csv(self, table_name, data):
        """
        Appends a single row of data to a CSV file.
        
        Args:
            table_name (str): The name of the log (e.g., 'training_results', 'eval_metrics').
            data (dict): Dictionary containing the metrics and parameters to log.
        """
        filename = f"{table_name}.csv"
        
        # Convert dictionary to DataFrame
        df = pd.DataFrame([data])
        
        # Check if file exists to determine if header is needed
        file_exists = os.path.isfile(filename)
        
        try:
            # Mode 'a' for append
            df.to_csv(filename, mode='a', header=not file_exists, index=False)
        except Exception as e:
            print(f"⚠️ Error logging to {filename}: {e}")

    def log_eval_metrics(self, model_id, metrics):
        """
        Helper specifically for the required metrics: Bleu, RougeL, Perplexity.
        """
        log_data = {
            "model_id": model_id,
            "bleu": metrics.get("bleu", 0),
            "rougeL": metrics.get("rougeL", 0),
            "perplexity": metrics.get("perplexity", 0),
            "timestamp": pd.Timestamp.now()
        }
        self.log_to_csv("evaluation_history", log_data)
        
    def log_experiment_summary(self, experiment_id, model_id, lora_config, train_loss, val_loss, metrics, training_time=None):
        """
        Logs overall experiment details to LLAMAExperiments.csv
        """
        log_data = {
            "id": experiment_id,
            "model_name": model_id,
            "lora_config": str(lora_config) if lora_config is not None else "Unsloth-optimized (r=16, alpha=32)",
            "train_loss": round(train_loss, 4) if train_loss is not None else None,
            "val_loss": round(val_loss, 4) if val_loss is not None else None,   # ← Fixed here
            "perplexity": metrics.get("perplexity", None),
            "bleu": metrics.get("bleu", None),
            "rouge_l": metrics.get("rouge_l", None),
            "training_time_seconds": round(training_time, 2) if training_time is not None else None,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        self.log_to_csv("LLAMAExperiments", log_data)

In [8]:
import torch
import evaluate
import pandas as pd
import re
from datetime import datetime
from tqdm import tqdm

# 5. EVALUATOR CLASS (Synced with Llama 3.1 Special Tokens)
class Evaluator:
    def __init__(self, model, tokenizer, logger, max_new_tokens):
        self.model = model
        self.tokenizer = tokenizer
        self.logger = logger
        self.max_new_tokens = max_new_tokens
        
        # Load metrics
        self.bleu = evaluate.load("bleu")
        self.rouge = evaluate.load("rouge")

    @torch.inference_mode()
    def calculate_metrics(self, test_dataset, experiment_id):
        """
        Evaluation loop optimized for the Llama 3.1 Instruct header format.
        """
        print(f"\n🧪 Evaluating {len(test_dataset)} samples...")
        
        predictions, references = [], []
        total_loss, total_tokens = 0.0, 0
        
        self.model.eval()
        
        # Prompt boundary identifier from your Preprocessor
        ASSISTANT_TAG = "<|start_header_id|>assistant<|end_header_id|>\n"
        eos_token_id = self.tokenizer.eos_token_id

        for idx, example in tqdm(enumerate(test_dataset), total=len(test_dataset), desc="Testing"):
            full_text = example['text']
            
            # Split input from expected output
            if ASSISTANT_TAG in full_text:
                parts = full_text.split(ASSISTANT_TAG)
                input_text = parts[0] + ASSISTANT_TAG
                expected = parts[1].split("<|end_of_text|>")[0].strip()
            else:
                continue

            inputs = self.tokenizer(input_text, return_tensors="pt").to(self.model.device)
            
            # --- Inference Generation ---
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                use_cache=True,
                do_sample=True,
                temperature=0.3,           # Low temp to keep Bengali response consistent
                top_p=0.9,
                repetition_penalty=1.1,   # Prevents loops without forcing English fallback
                pad_token_id=eos_token_id,
                eos_token_id=eos_token_id
            )
            
            # Decode only the newly generated tokens
            gen_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
            gen_text = self.tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
            
            # Post-generation cleanup
            gen_text = gen_text.split("<|")[0].strip()
            gen_text = gen_text.split("User:")[0].strip()

            # --- Perplexity Logic ---
            enc_full = self.tokenizer(full_text, return_tensors="pt").to(self.model.device)
            enc_input = self.tokenizer(input_text, return_tensors="pt").to(self.model.device)
            
            labels = enc_full["input_ids"].clone()
            labels[:, :enc_input["input_ids"].shape[1]] = -100 
            
            loss_outputs = self.model(**enc_full, labels=labels)
            num_valid_tokens = (labels != -100).sum().item()
            
            if num_valid_tokens > 0:
                total_loss += loss_outputs.loss.item() * num_valid_tokens
                total_tokens += num_valid_tokens

            predictions.append(gen_text)
            references.append(expected)
            
            # Individual result logging
            self.logger.log_to_csv("GeneratedResponses", {
                "id": f"TEST_{idx+1:04d}", 
                "experiment_id": experiment_id,
                "input_text": input_text.strip(),
                "expected": expected,
                "generated": gen_text,
                "status": "Bengali" if any('\u0980' <= c <= '\u09FF' for c in gen_text) else "English/Other"
            })

        # --- Compute Final Metrics ---
        print("\n📊 Calculating final scores using sub-word tokenization...")
        
        def llama_tokenizer_fn(text):
            return self.tokenizer.convert_ids_to_tokens(self.tokenizer.encode(text, add_special_tokens=False))

        try:
            b_score = self.bleu.compute(predictions=predictions, references=[[r] for r in references], tokenizer=llama_tokenizer_fn)['bleu']
        except: b_score = 0.0
        
        try:
            r_score = self.rouge.compute(predictions=predictions, references=references, tokenizer=llama_tokenizer_fn)['rougeL']
        except: r_score = 0.0
        
        avg_loss = total_loss / total_tokens if total_tokens > 0 else 0
        metrics = {
            "perplexity": round(torch.exp(torch.tensor(avg_loss)).item(), 4),
            "bleu": round(b_score, 4),
            "rouge_l": round(r_score, 4)
        }
        
        self.logger.log_to_csv("EvaluationSummary", {
            "experiment_id": experiment_id,
            "perplexity": metrics["perplexity"],
            "bleu": metrics["bleu"],
            "rouge_l": metrics["rouge_l"],
            "timestamp": datetime.now()
        })
        
        return metrics


In [9]:
import torch
from datetime import datetime
from trl import SFTConfig, SFTTrainer
from unsloth import FastLanguageModel

# 6. MAIN TUNER CLASS
class LLAMAFineTuner:
    """
    Handles fine-tuning using the Llama 3.1 Instruct format.
    """
    def __init__(self, strategy, model_id, logger):
        self.strategy = strategy
        self.model_id = model_id
        self.logger = logger
        self.model = None
        self.tokenizer = None

    def run_training(self, train_dataset, val_dataset, max_seq_length):
        """
        Executes SFT training with epoch-based evaluation.
        """
        # 1. Setup model and tokenizer
        self.model, self.tokenizer, _ = self.strategy.setup_model(self.model_id, max_seq_length)
        
        # 2. Configuration for SFT
        training_args = SFTConfig(
            output_dir="/kaggle/working/checkpoints",
            per_device_train_batch_size=4,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=2,
            learning_rate=2e-4,
            num_train_epochs=1,
            fp16=not torch.cuda.is_bf16_supported(),
            bf16=torch.cuda.is_bf16_supported(),
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            gradient_checkpointing=True,
            logging_steps=5,
            eval_strategy="epoch",      # Trigger at the end of each epoch
            save_strategy="no",
            report_to="none",
            dataset_text_field="text",
            max_seq_length=max_seq_length,
        )
        
        # 3. Initialize Trainer
        trainer = SFTTrainer(
            model=self.model,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            processing_class=self.tokenizer,
            args=training_args,
        )

        # 4. Train
        print("\n🚀 Training with Llama 3.1 Instruct Format...")
        start_time = datetime.now()
        train_result = trainer.train()
        training_time = (datetime.now() - start_time).total_seconds()
        
        # 5. Final Evaluation Check
        # If eval_strategy="epoch" didn't trigger (e.g., small dataset), we force it here
        print("\n📊 Finalizing validation metrics...")
        eval_metrics = trainer.evaluate()
        final_val_loss = eval_metrics.get("eval_loss", 0)
        
        # 6. Save model and tokenizer
        print(f"\n💾 Saving fine-tuned weights... (Final Val Loss: {final_val_loss:.4f})")
        self.model.save_pretrained("/kaggle/working/final_model")
        self.tokenizer.save_pretrained("/kaggle/working/final_model")
        
        # 7. Cleanup
        print("\n🧹 Cleaning up VRAM...")
        del self.model
        del trainer
        torch.cuda.empty_cache()
        
        return train_result, final_val_loss, training_time

In [10]:
# 7. AUTHENTICATION FUNCTION
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

def authenticate_huggingface():
    """Authenticate with Hugging Face using Kaggle secrets."""
    try:
        user_secrets = UserSecretsClient()
        token = user_secrets.get_secret("HF_TOKEN")
        login(token=token)
        print("✅ Logged in to Hugging Face successfully!")
    except Exception as e:
        print(f"❌ Could not find HF_TOKEN in Kaggle Secrets: {e}")
        raise


In [12]:
# --- TRAINING EXECUTION CELL ---
# This cell handles the orchestration of the fine-tuning process.
# It calculates dataset lengths before loading the model to save time and memory.

from pathlib import Path
import json  # <--- ADD THIS LINE HERE
import torch
# 1. Configuration & Authentication
MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
DATASET_PATH = "/kaggle/input/bengali-empathetic-conversations-corpus/BengaliEmpatheticConversationsCorpus .csv"
EXPERIMENT_ID = "EXP_001"

# Authenticate Hugging Face
authenticate_huggingface()

# 2. Optimized Length Calculation
# We load just the tokenizer first to avoid a heavy 5GB model load during processing
print("🔄 Initializing tokenizer for dataset processing...")
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# 3. Dataset Processing
print("📊 Processing dataset and calculating sequence lengths...")
processor = DatasetProcessor(DATASET_PATH, tokenizer)
train_ds, val_ds, test_ds, absolute_max_len, new_max_token = processor.process_and_get_max_len()

# 4. Persistence for Inference Cell
# We save the calculated lengths so the next cell (Inference) can read them 
# without needing to re-process the dataset.
config_metadata = {
    "max_seq_length": absolute_max_len,
    "max_new_tokens": new_max_token,
    "model_id": MODEL_ID,
    "experiment_id": EXPERIMENT_ID
}

with open("/kaggle/working/exp_config.json", "w") as f:
    json.dump(config_metadata, f)

# Save test dataset for the separate Inference cell
test_ds.save_to_disk("/kaggle/working/test_dataset")
print(f"✅ Metadata and Test Dataset saved to /kaggle/working/")

# 5. Fine-Tuning Execution
try:
    # Initialize components
    strategy = UnslothStrategy()
    logger = ExperimentLogger()
    tuner = LLAMAFineTuner(strategy, MODEL_ID, logger)
    
    # Run the training loop
    # The tuner internally handles model loading, training, and memory cleanup
    results, val_loss, training_time = tuner.run_training(
        train_dataset=train_ds,
        val_dataset=val_ds,
        max_seq_length=absolute_max_len
    )
    
    print("\n" + "="*30)
    print("🎉 TRAINING SUCCESSFUL")
    print(f"📈 Final Training Loss: {results.training_loss:.4f}")
    print(f"📈 Final Validation Loss: {val_loss:.4f}")
    print(f"⏱️ Total Time: {training_time:.2f} seconds")
    print("="*30)
    # Log full experiment summary
    logger.log_experiment_summary(
        experiment_id=EXPERIMENT_ID,
        model_id=MODEL_ID,
        lora_config=None,  # UnslothStrategy returns None for lora_config
        train_loss=results.training_loss,
        val_loss=val_loss,
        metrics={},  # We'll update full metrics after evaluation
        training_time=training_time
    )
    print("🚀 You can now proceed to the Inference & Evaluation cell.")

except Exception as e:
    print(f"\n❌ Training failed: {str(e)}")
    # Explicit memory cleanup in case of crash
    if 'tuner' in locals(): del tuner
    torch.cuda.empty_cache()
    raise


✅ Logged in to Hugging Face successfully!
🔄 Initializing tokenizer for dataset processing...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

📊 Processing dataset and calculating sequence lengths...


...

Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,0.417800,0.387612


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient



📊 Finalizing validation metrics...



💾 Saving fine-tuned weights... (Final Val Loss: 0.3876)

🧹 Cleaning up VRAM...

🎉 TRAINING SUCCESSFUL
📈 Final Training Loss: 0.4512
📈 Final Validation Loss: 0.3876
⏱️ Total Time: 19652.64 seconds
🚀 You can now proceed to the Inference & Evaluation cell.


In [13]:
# --- TESTING & INFERENCE EXECUTION ---
# Run this cell after training is complete to evaluate the fine-tuned model.

import json
import torch
from datetime import datetime
from datasets import load_from_disk
from unsloth import FastLanguageModel

if __name__ == "__main__":
    EXPERIMENT_ID = "EXP_001"
    CONFIG_PATH = "/kaggle/working/exp_config.json"
    DATASET_SAVE_PATH = "/kaggle/working/test_dataset"
    MODEL_PATH = "/kaggle/working/final_model"
    
    try:
        # 1. Load Configuration from Training
        print(f"📂 Loading experiment configuration...")
        with open(CONFIG_PATH, "r") as f:
            config = json.load(f)
        
        saved_max_len = config["max_seq_length"]
        saved_new_tokens = config["max_new_tokens"]
        
        # 2. Load the saved test dataset
        test_ds = load_from_disk(DATASET_SAVE_PATH)
        print(f"✅ Loaded {len(test_ds)} test samples")

        # 3. Load Model for Inference
        # We load directly here to avoid the AttributeError from the Tuner class
        print(f"💾 Loading fine-tuned model from {MODEL_PATH}...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=MODEL_PATH,
            max_seq_length=saved_max_len,
            dtype=None,
            load_in_4bit=True,
        )
        FastLanguageModel.for_inference(model)

        # 4. Run Evaluation using the Evaluator class
        logger = ExperimentLogger()
        evaluator = Evaluator(model, tokenizer, logger, max_new_tokens=saved_new_tokens)
        
        test_metrics = evaluator.calculate_metrics(
            test_dataset=test_ds, 
            experiment_id=EXPERIMENT_ID
        )

        # 5. Final Logging
        print("\n" + "="*30)
        print("🎉 EVALUATION COMPLETE")
        print(f"📊 Perplexity: {test_metrics['perplexity']:.4f}")
        print(f"📊 BLEU:       {test_metrics['bleu']:.4f}")
        print(f"📊 ROUGE-L:    {test_metrics['rouge_l']:.4f}")
        print("="*30)
        # Update the experiment log with final evaluation metrics
        logger.log_experiment_summary(
            experiment_id=EXPERIMENT_ID,
            model_id=MODEL_ID,
            lora_config=None,
            train_loss=None,  # Already logged in training phase
            val_loss=None,
            metrics=test_metrics,
            training_time=None
        )

        # Cleanup memory
        del model
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"❌ Testing failed: {str(e)}")
        if 'model' in locals(): del model
        torch.cuda.empty_cache()
        raise

📂 Loading experiment configuration...
✅ Loaded 198 test samples
💾 Loading fine-tuned model from /kaggle/working/final_model...
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/meta-llama-3.1-8b-instruct-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.



🧪 Evaluating 198 samples...


Testing: 100%|██████████| 198/198 [30:02<00:00,  9.11s/it] 



📊 Calculating final scores using sub-word tokenization...

🎉 EVALUATION COMPLETE
📊 Perplexity: 1.8306
📊 BLEU:       0.2276
📊 ROUGE-L:    0.3598


In [14]:
import shutil
import os

zip_name = "kaggle_working_all.zip"
work_dir = "/kaggle/working"

# Remove old zip if exists
if os.path.exists(zip_name):
    os.remove(zip_name)

# Create zip
shutil.make_archive(
    base_name=zip_name.replace(".zip", ""),
    format="zip",
    root_dir=work_dir
)

print(f"✅ Created {zip_name}")
print("📥 You can now download it from the right panel (Output files).")


✅ Created kaggle_working_all.zip
📥 You can now download it from the right panel (Output files).
